# C++ Builder & Runner

In [ ]:
subprocess.check_call([sys.executable, "-m", "pip", "install", "requests"])

%run .builder/build.py
%run .builder/tools/format.py
%run .builder/tools/documentation.py
%run .builder/project_templates/TDD.py

In [ ]:
# project_tdd_setup('.')

In [ ]:
configs = [
    # Обычный Debug
    # {
    #     'name': 'GCC',   
    #     'cc': 'g++',     
    #     'type': 'debug', 
    #     # Добавил -std=c++23
    #     'flags': '-std=c++23 -g -O0'
    # },
    # # Обычный Release
    # {
    #     'name': 'CLANG', 
    #     'cc': 'clang++', 
    #     'type': 'release',
    #     # Добавил -std=c++23
    #     'flags': '-std=c++23 -O3 -Wall'
    # },
    # Coverage Configuration
    {
        'name': 'GCC-Cov',
        'cc': 'g++',
        'type': 'test',
        # Добавил -std=c++23
        'flags': '-std=c++23 -g -O0',
        'cov_flags': '--coverage',
        'cov_tool': 'lcov'
    }
]

for c in configs: run_pipeline(c)

In [ ]:
# gen_docs()

In [ ]:
# format_code()

In [ ]:
# clean()